# Introductory Overview of PyMC

## A Motivating Example: Linear Regression

This notebook introduces model

- Definition
- Fitting
- Posterior analysis

For this introduction, we consider a simple Bayesian linear regression
model with normally-distributed priors for the parameters. Specifically,
we are interested in predicting outcomes, $Y$, as normally distributed
observations with an expected value, $\mu$, that is a linear function of
two predictor variables, $X_{1}$ and $X_{2}$:

$$
\begin{gather}
Y \sim \mathcal{N}(\mu, \sigma^2) \\
\mu = \alpha + \beta_{1} X_{1} + \beta_{2} X_{2}
\end{gather}
$$

where $\alpha$ is the intercept, $\beta_{i}$ is the coefficient
for covariate $X_{i}$, and $\sigma$ represents the observation error.

Since we are constructing a Bayesian model, we must assign a
prior **distributions** for each unknown variable in the model.

We choose zero-mean normal priors with a variance of 100 for both
regression coefficients. This choice corresponds to **weak**
information about the **true** parameter values. Additionally, we
choose a half-normal distribution as the prior for $\sigma$.

$$
\begin{gather}
\alpha \sim \mathcal{N}(0, 100) \\
\beta_{i} \sim \mathcal{N}(0, 100) \\
\sigma \sim | \mathcal{N}(0, 1) |
\end{gather}
$$

### Generating data

We can simulate some artificial data using `numpy.random`.
After simulating artificial data, we will use PyMC to try
to recover the corresponding parameters.

Intentionally, we are generating data to closely corresponding
to the PyMC model structure.

#### Initialize our "environment"

In [ ]:
# Import the necessary packages
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Configure the retina inline backend
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Initialize our repeatable random number generator
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

In [ ]:
# Use a dark grid for visualization
az.style.use('arviz-darkgrid')

#### Initialize the (simulated) "actual" data

In [ ]:
# True parameter values
alpha = 1
sigma = 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variables
X = np.random.randn(size), np.random.randn(size) * 0.2

# Simulate the observed outcome variables
Y = alpha + beta[0] * X[0] + beta[1] * X[1] + rng.normal(size=size) * sigma

Let's visualize the (simualted) "real" data.

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10, 4))
axes[0].scatter(X[0], Y, alpha=0.6)
axes[1].scatter(X[1], Y, alpha=0.6)
axes[0].set_ylabel('Y')
axes[0].set_xlabel('X0')
axes[1].set_xlabel('X1')
plt.show()